<a href="https://colab.research.google.com/github/AV-Gorshkov/Python_netology/blob/main/PY_%D0%BB%D0%B8%D0%BD%D0%B5%D0%B9%D0%BD%D0%B0%D1%8F_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D1%8F(%D1%87%D0%B0%D1%81%D1%82%D1%8C_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предсказываем стоимость мед страховки



## Загужаем необходимые библиотеки

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Загружаем данные

Собраны данные:
- age: возраст
- sex: пол
- bmi: индекс массы тела
- children: количество детей, охваченных медицинским страхованием / количество иждивенцев
- smoker: курение
- region: регион (northeast, southeast, southwest, northwest).
- charges: индивидуальные медицинские расходы (его и хотим предсказать)

In [ ]:
# Загрузите данные из файла insurance.csv в переменную df
import pandas as pd
df=pd.read_csv( 'insurance.csv', sep=',')

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Преобразуем строковые данные

In [ ]:
# Заменяем пол и курение на числа
df['sex']=df['sex'].map({'male':1, 'female':0})
df['smoker']=df['smoker'].map({'yes':1,'no':0})

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [ ]:
# Заменяем регион на набор отдельных колонок (is_southwest, is_southeast и тп)
df = pd.get_dummies(df, columns=['region'])
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


## Формируем признаки и целевую переменную

In [ ]:
## Сформируйте признаки и целевую переменную
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges',
       'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest'],
      dtype='object')

In [ ]:
# X, y = ...
X = df[['age', 'sex', 'bmi', 'children', 'smoker','region_northeast', 'region_northwest', 'region_southeast','region_southwest']]
y = df['charges']

In [ ]:
X

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,0,0,0,1
1,18,1,33.770,1,0,0,0,1,0
2,28,1,33.000,3,0,0,0,1,0
3,33,1,22.705,0,0,0,1,0,0
4,32,1,28.880,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,0,1,0,0
1334,18,0,31.920,0,0,1,0,0,0
1335,18,0,36.850,0,0,0,0,1,0
1336,21,0,25.800,0,0,0,0,0,1


In [ ]:
y

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64

## Разделяем данные на выборку для обучения/проверки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.23)

## Создаем и обучаем модель линейной регресии

In [ ]:
# Создайте и обучите модель
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

## Получаем предсказание и оцениваем качество

In [ ]:
# Получите предсказание
lr.coef_

array([  256.38177364,  -200.54241774,   339.56855191,   386.62844085,
       24160.14945615,   757.85138246,   213.43256284,  -560.94147974,
        -410.34246555])

In [ ]:
# Оцените качество, при помощи метода mean_squared_error для тестовой выборки

In [ ]:
lr.predict(X_test)

array([28087.94346412, 12697.44126566,  1627.60292907, 10565.97356513,
       14367.8959404 , 33660.98905539,  4416.97721177,  2777.2152554 ,
       24427.54634493,  6590.16332541, 14635.24550231, 10978.6582376 ,
        1021.19111238,  3483.15617706, 27096.54400134, 31553.30532891,
       16310.55366615, 11965.72894275,  6313.49764198, 26455.16985089,
        3409.95948324, -1346.82061359, 12592.81947667,  8162.4431193 ,
        7184.6150094 , 11090.03986221,  -188.92356251,  7405.93106861,
       11442.19726562, 27312.26387982,  7742.47221759,  1274.16216661,
       30945.49938711,  1366.01766903,  1875.65323313, 27817.03671618,
       10354.86202424,  5772.25662811, 11698.72550588, 16130.66502002,
       13239.18618187, 12808.59047964, 14892.87386806,  4225.01045487,
       10333.36989177,  5200.29183041, 33700.81032752, 11580.78206089,
       13184.33040356, 13899.91107299, 29137.27794203,  6541.40194903,
       29352.8477092 ,  6535.34736091,  8023.74646618,  9280.91938683,
      

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, lr.predict(X_test))

39682967.00889356

In [ ]:
mean_squared_error(y_train, lr.predict(X_train))

35600829.42130369

## Делаем предсказание для одного человека

In [ ]:
# Заполняем данные по конкретному человеку
data = [{
    "age": 20,
    "sex": 1,
    "bmi": 30,
    "children": 2,
    "smoker": 1,
    "region_northeast": 0,
    "region_northwest": 0,
    "region_southeast": 1,
    "region_southwest": 0
}]

In [ ]:
df_person = pd.DataFrame(data)
df_person.head()

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,20,1,30,2,1,0,0,1,0


In [ ]:
lr.predict(df_person)

array([27026.23321692])

**Задание_2 (со звёздочкой):**

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

df_cost=pd.read_csv( 'CSV_(Google).csv', sep=';')
df_cost.head(7)

,траты в позапрошлом месяце,траты в прошлом месяце,траты в текущем месяце,траты в следующем месяце
0,50,100,100,100
1,200,200,150,150
2,250,200,150,200
3,300,200,250,200
4,300,200,300,200
5,300,300,400,200
6,400,300,400,300


In [ ]:
df_cost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   траты в позапрошлом месяце  12 non-null     int64
 1   траты в прошлом месяце      12 non-null     int64
 2   траты в текущем месяце      12 non-null     int64
 3   траты в следующем месяце    12 non-null     int64
dtypes: int64(4)
memory usage: 512.0 bytes


In [ ]:
df_cost.describe()

,траты в позапрошлом месяце,траты в прошлом месяце,траты в текущем месяце,траты в следующем месяце
count,12.000000,12.000000,12.000000,12.000000
mean,395.833333,420.833333,537.500000,416.666667
std,234.964536,386.980346,481.534007,386.319071
min,50.000000,100.000000,100.000000,100.000000
25%,287.500000,200.000000,225.000000,200.000000
50%,350.000000,300.000000,400.000000,250.000000
75%,500.000000,425.000000,550.000000,500.000000
max,1000.000000,1500.000000,1500.000000,1500.000000


In [ ]:
df_cost.columns

Index(['траты в позапрошлом месяце', 'траты в прошлом месяце',
       'траты в текущем месяце', 'траты в следующем месяце'],
      dtype='object')

In [ ]:
X = df_cost[['траты в позапрошлом месяце', 'траты в прошлом месяце', 'траты в текущем месяце']]
y = df_cost['траты в следующем месяце']

In [ ]:
X
# y

,траты в позапрошлом месяце,траты в прошлом месяце,траты в текущем месяце
0,50,100,100
1,200,200,150
2,250,200,150
3,300,200,250
4,300,200,300
5,300,300,400
6,400,300,400
7,450,350,500
8,500,400,500
9,500,500,700


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
lr.coef_

array([ 0.40200642,  0.85888535, -0.08706776])

In [ ]:
lr.predict(X_test)

array([323.17484067, 377.51265281, 509.03222378])

In [ ]:
mean_squared_error(y_test, lr.predict(X_test))

1957.689936226531

In [ ]:
mean_squared_error(y_train, lr.predict(X_train))

1708.739525346731

MSE для обучаемой выборки = 1708 (т.е. примерно  ± 41р. от стоимости покупки), для тестовой выборки MSE = 1957 (примерно ± 44р. от стоимости покупки).Расхождение между MSE для тестовой и для обучаемой выборки незначительное, т.е. модель может прогнозировать затраты на будующий период.